In [1]:
import cv2     # for capturing videos
import os 
import shutil
import math   # for mathematical operations
import pandas as pd
import pickle
import numpy as np   
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from glob import glob
from tqdm import tqdm
import tensorflow as tf 
from tensorflow.keras.preprocessing import image   # for preprocessing the images
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from ax.service.ax_client import AxClient
from ax.utils.notebook.plotting import render, init_notebook_plotting
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12524848515073135752
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7272764544
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15362937346370719963
physical_device_desc: "device: 0, name: GeForce RTX 2080 SUPER, pci bus id: 0000:0a:00.0, compute capability: 7.5"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 7596800736
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11862271551827210110
physical_device_desc: "device: 1, name: GeForce RTX 2080 SUPER, pci bus id: 0000:0b:00.0, compute capability: 7.5"
]


In [3]:
with tf.device('/device:GPU:0'):
    if tf.test.gpu_device_name():
        print("GPU")
    else:
        print("no GPU")

GPU


In [12]:
def img_to_array(path, df_csv):
    with tf.device('/device:GPU:0'):
        if tf.test.gpu_device_name():
            print("Using GPU")
        base_model = tf.keras.applications.ResNet50(weights='imagenet', pooling='avg', include_top = False) 
        for layer in base_model.layers:
            layer.trainable = False

        list_image = []
    
        # for loop to read and store frames
        for i in tqdm(range(df_csv.shape[0])):
            # loading the image and keeping the target size as (224,224,3)
            img = image.load_img(path + df_csv['image'][i], target_size=(224,224,3))
            # converting it to array
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            features = base_model.predict(x)
            features = features.squeeze()

            # appending the image to the train_image list
            list_image.append(features)
          
        X = np.array(list_image)
        y = df_csv['class']
    
    return X, y

In [19]:
train = pd.read_csv('train_new.csv')
X, y= img_to_array('data/', train)

Using GPU


100%|██████████| 80569/80569 [59:12<00:00, 22.68it/s]  


In [20]:
with open('pickle/X.pickle', 'wb') as f:
    pickle.dump(X, f)
    
with open('pickle/y.pickle', 'wb') as f:
    pickle.dump(y, f)